In [41]:

import nltk.data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
import types
import pandas as pd

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hilal.pataci/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/hilal.pataci/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [42]:
#txt='../sec_cleaning/txt/' #path of txt files from SEC-cleaned


In [43]:
import glob,os
from pathlib import Path
#from google.colab import drive
#drive.mount('/content/gdrive')

#print(os.getcwd())

#os.chdir("../sec_cleaning/txt")
#print(os.getcwd())
#breakpoint()
rid = 0
sentences = {}
filenames=[]
#read file path

prefixed_files = [filename for filename in glob.glob("./txt/*.txt") if filename.startswith("./txt/AAL")]
#print(prefixed_files)
#for filename in glob.glob("./txt/*.txt"):
#    print(filename)
for text_path in prefixed_files:
    #print("foo")
    #read all txt file 
    text = Path(text_path).read_text()
    filenames.append(os.path.basename(text_path))
    #breakpoint()
    #print(text)
    #breakpoint()
    #create a dictionary and assign each sentence to a dictionary.
    #dictionary will hold the sentences of each file
    sentences[rid] = []
    text = text.strip()
    #extract each sentence and assign to corresponding the dictionary
    sentences[rid].extend(sent_detector.tokenize(text))
    rid+=1

print(len(sentences))
#print(sentences[0])
#print(sentences[1])

10


In [44]:
import re
causal_verbs = [
    "cause", "causes","caused",
    "result in", "results in", "resulted in", 
    "induce", "induces", "induced", 
    "give rise", "gives rise", "gave rise", "given rise", 
    "generate", "generates", "generated", 
    "bring about", "brings about", "brought about", 
    "lead to", "leads to", "led to", 
    "trigger", "triggers", "triggered", 
    "is linked to", "are linked to", "were linked to", "was linked to", 
    "brings forth", 
    "leads up to", "lead up to", "led up to",
    "triggers off", "trigger off", "triggered off", 
    "bring on", "brings on", "brought on",
    "affect", "affects", "affected",
    "effect", "effects", "effected",
    "instigate", "instigates", "instigated",
    "stimulate", "stimulated", "stimulates",
    "engender", "engendered","engenders", 
    "impact", "impacts", "impacted",
    "become ground", "became ground", "become grounds", "became grounds"
]
reverse_causal_verbs = [
    "stems from", "stemmed from", "stem from",
    "brought on by", 
    "caused by", 
    "affected by", "effected by",
    "resulted from", "results from", "as a result of",
    "emerged from", "emerges from", "carried out by", "invoked by", "occurred because", "in consequence"
]
causal_re = re.compile(" | ".join(causal_verbs), re.IGNORECASE)
reverse_causal_re = re.compile(" | ".join(reverse_causal_verbs), re.IGNORECASE)

In [45]:
use_chunks = False
chunking_method = "nltk"
min_chunk_strlen = 6

# The following is a temporary simple definition of get_ce
def get_ce(part1, part2, use_chunks=False, chunking_method="nltk", min_chunk_strlen=6):
    pairs = []
    pairs.append((part1,part2))
    return pairs
    
def get_ce_pairs(sent, rec_id, use_chunks=False, chunking_method="nltk", min_chunk_strlen=6):
    cees = []
    sent = re.sub('[\\s+]', ' ', sent)
    #if len(sent) > 200:
    #    return None
    parts = causal_re.split(sent)
    for i in range(0, len(parts) - 1):
        for (cause, effect) in get_ce(parts[i],parts[i+1],use_chunks, chunking_method, min_chunk_strlen):
            cees.append((cause, effect,rec_id, sent))
    parts = reverse_causal_re.split(sent)
    for i in range(0, len(parts) - 1):
        for (cause, effect) in get_ce(parts[i+1],parts[i],use_chunks, chunking_method, min_chunk_strlen):
            cees.append((cause, effect,rec_id, sent))
    return cees

In [46]:
"""
sid = 8
print(len(sentences))
sent = sentences[sid][10]
print(sent)
get_ce_pairs(sent, sid)
sent_pos = nltk.pos_tag(nltk.word_tokenize(sent))
sent_pos
"""

'\nsid = 8\nprint(len(sentences))\nsent = sentences[sid][10]\nprint(sent)\nget_ce_pairs(sent, sid)\nsent_pos = nltk.pos_tag(nltk.word_tokenize(sent))\nsent_pos\n'

In [47]:
grammar = r"""
          NP: {<NN|NNS>+<><NN|NNS>+}              # Verb and its suroundings
          """

grammar = r"""
  NP: {<PDT|DT|JJ|NN.*|\(|\)>+}          # Chunk sequences of DT, JJ, NN
  PP: {<IN|DT><NP>}               # Chunk prepositions followed by NP
  VP: {<VB.*><NP|PP|CLAUSE>+(<\.>)?$} # Chunk verbs and their arguments
  CLAUSE: {<NP><VP>}           # Chunk NP, VP
  """
cp = nltk.RegexpParser(grammar, loop=1)

def get_verb_left_right(sent):
    verb = None
    left_np = None
    right_np = None
    sent_pos = nltk.pos_tag(nltk.word_tokenize(sent))
    parsed_sent = cp.parse(sent_pos)
    for node in parsed_sent:
        if node.__class__.__name__=='tuple':
            continue
        if node.label()=='NP':
            left_np = node
        if node.label()=='VP':  
            vp = node
            verb = vp[0][0]
            if len(vp)>1:
                if vp[1].__class__.__name__!='tuple' and vp[1].label()=='PP':
                    verb += " " + vp[1][0][0]
            for vnode in vp[1]:
                if vnode.__class__.__name__=='tuple':
                    continue
                if vnode.label()=='NP':
                    right_np = vnode
    if verb and right_np and left_np:
        return (verb,' '.join(v[0] for v in left_np.leaves()),' '.join(v[0] for v in right_np.leaves()))
    else:
        return None

In [48]:
import csv
Sentence_output_hilal_2 = "Sentence_outputhilal_dummy.csv"
csvOutput = open(Sentence_output_hilal_2,"w") #<===open and write to a csv file  and New rows will be appended.
csvWriter = csv.writer(csvOutput, quoting = csv.QUOTE_NONNUMERIC)
csvWriter.writerow(["file name","direction","verb","cause","effect","sent"])


count = 0
#breakpoint()
for rid in sentences:
    for sent in sentences[rid]:
        parts = get_verb_left_right(sent)
        if parts:
            verb = parts[0]
            if verb.lower() in causal_verbs:
                count += 1
                cause = parts[1]
                effect = parts[2]
                print("Found causal verb and cause-effect pairs: ({},{},{}) from sentence: {}".format(verb,cause,effect,sent))
                csvWriter.writerow([filenames[rid],"forward",verb,cause,effect,sent])
            if verb.lower() in reverse_causal_verbs:
                count += 1
                cause = parts[2]
                effect = parts[1]
                csvWriter.writerow([filenames[rid],"reverse",verb,cause,effect,sent])

                print("Found reverse causal verb and cause-effect pair: ({},{},{}) from sentence: {}".format(verb,cause,effect,sent))        
                print("Found {} cause-effect pairs.".format(count))
                

print("end of loop")

csvOutput.close()


TypeError: can only concatenate str (not "tuple") to str